<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:



In [4]:
import folium
import pandas as pd

In [5]:
from folium.plugins import MarkerCluster 
from folium.plugins import MousePosition
from geopy.distance import geodesic
from folium.features import DivIcon

Let me explain these **Folium plugins** in a fun and simple way, like you're exploring a magical map! 🗺️

---

### **1. MarkerCluster**
Imagine you have a treasure map with lots of tiny X's marking treasure spots. If all the X's are close together, the map might look messy. The **MarkerCluster** plugin groups those X's into one big pile. When you zoom in, the pile breaks apart, showing the individual X's (treasures). 

It’s like keeping your toys in one box and taking them out one by one when you play closer!

---

### **2. MousePosition**
This plugin is like having a magical magnifying glass that tells you exactly where your mouse pointer is on the map. Wherever you move the mouse, it shows secret coordinates (like map addresses) at the bottom of the screen. It’s super handy if you want to know the exact spot of a treasure!

---

### **3. DivIcon**
This is a fancy way to add cool custom markers on your map. Instead of boring old dots, you can use words, emojis, or even fun shapes to mark a spot. It’s like decorating your treasure map with colorful stickers to make isy.


## To do
citing and ea
s
yFirst, let's try to add each site's location on a map using site's latitude and longitude coordinates
The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


s this make sense? 😊

In [7]:
spacex_df = pd.read_csv('spacex_launch_geo.csv')
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


What we did above are three things:
1. We selected the relevant columns from our main dataframed (spacex_df) that we need for our analysis- of creating a map to display launch sites, their coordintaes and class (a column that states the outcome of whether or not the launch was successful. and saved it in a new dataframe called
2. Then we grouped the selected columns together but based on each launch site, ie putting laucnhes from the same site together, as several rockets were launched from the same location. Sowe have to group them so as to get an orderly data

- spacex_df is the original table with all the rocket launch data.
- .groupby(['Launch Site']) tells the computer to gather the data based on each launch site, putting launches from the same site into groups.
- as_index=False makes sure that the grouped data doesn’t use the "Launch Site" column as a special index, which helps keep the data easy to read.
- .first() tells the computer to grab the first row of data from each group.

  
Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

Why? a folium object is neccessary because it tells the map where **focus** as a starting point. the object has a pair of lat and long. so that readers don't have to search mannually. Again, it needs a **zoom** level to decide how much of the starting point  needs to show. 12 is detailed and 3 is broader

In [10]:
# Start location is NASA Johnson Space Center

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name

circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    
    # Create an icon as a text label
    icon = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


In [14]:
# Define a map centered at a specific location
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# List of locations and their details
locations = [
    {"coords": [28.562302, -80.577356], "radius": 2200, "color": "#d35400", "name": "CCAFS LC-40"},
    {"coords": [28.563197, -80.576820], "radius": 2200, "color": "#d35400", "name": "CCAFS SLC-40"},
    {"coords": [28.573255, -80.646895], "radius": 2200, "color": "#d35400", "name": "KSC LC-39A"},
    {"coords": [34.632834, -120.610745], "radius": 2200, "color": "#d35400", "name": "VAFB SLC-4E"}
]

# Add circles and tooltips for each location
for loc in locations:
    # Add circle
    folium.Circle(
        location = loc["coords"],       # Circle's center
        radius = loc["radius"],         # Radius in meters
        color = loc["color"],           # Circle outline color
        fill = True,                    # Fill the circle
        fill_color = loc["color"],      # Fill color matches the outline
    ).add_to(site_map)


    icon = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )

    # Add tooltip for the location name
    folium.Marker(
        location=loc["coords"],
        tooltip=loc["name"]           # Tooltip that shows location name
    ).add_to(site_map)

# Display the map
site_map


In [15]:
site_map.save("site_map.html")

Now, you can explore the map by zoom-in/out the marked areas, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

# Explanation

The sites are in close proximities to the equator line and the coast as well.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [19]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object



In [21]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [23]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def assign_marker_color(launch_class):
    return "yellow" if launch_class == 1 else "red"

# Apply the function to create the marker_color column
spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)

# Display the updated dataframe
print(spacex_df)

     Launch Site        Lat        Long  class marker_color
0    CCAFS LC-40  28.562302  -80.577356      0          red
1    CCAFS LC-40  28.562302  -80.577356      0          red
2    CCAFS LC-40  28.562302  -80.577356      0          red
3    CCAFS LC-40  28.562302  -80.577356      0          red
4    CCAFS LC-40  28.562302  -80.577356      0          red
5    CCAFS LC-40  28.562302  -80.577356      0          red
6    CCAFS LC-40  28.562302  -80.577356      0          red
7    CCAFS LC-40  28.562302  -80.577356      0          red
8    CCAFS LC-40  28.562302  -80.577356      0          red
9    CCAFS LC-40  28.562302  -80.577356      0          red
10   CCAFS LC-40  28.562302  -80.577356      0          red
11   CCAFS LC-40  28.562302  -80.577356      0          red
12   CCAFS LC-40  28.562302  -80.577356      0          red
13   CCAFS LC-40  28.562302  -80.577356      0          red
14   CCAFS LC-40  28.562302  -80.577356      0          red
15   CCAFS LC-40  28.562302  -80.577356 

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [25]:
# Add marker_cluster to current site_map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon = folium.Icon(color='white', icon_color = row['marker_color']

# Initialize a folium map centered at an approximate location
site_map = folium.Map(location=[28.5, -80.6], zoom_start=3)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through the dataframe and add markers to the cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location = [row["Lat"], row["Long"]],
        icon = folium.Icon(color=row["marker_color"]),
        tooltip = f"Site: {row['Launch Site']}<br>Class: {row['class']}"
    ).add_to(marker_cluster)

#marker_cluster.add_child((marker)
# Display the map
site_map

C:\Users\Atabo Mohammed\AppData\Local\Temp\ipykernel_16680\3147188614.py:19: UserWarning: color argument of Icon should be one of: {'orange', 'black', 'gray', 'beige', 'pink', 'white', 'darkblue', 'darkred', 'darkpurple', 'cadetblue', 'green', 'lightblue', 'purple', 'lightred', 'darkgreen', 'lightgray', 'lightgreen', 'red', 'blue'}.
  icon = folium.Icon(color=row["marker_color"]),


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)



In [28]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
# Define a function to assign marker colors
def assign_marker_color(launch_class):
    return "green" if launch_class == 1 else "red"  # Folium expects lowercase color names

# Apply the function to create the marker_color column
spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)

# Initialize a folium map centered at an approximate location
site_map = folium.Map(location=[28.5, -80.6], zoom_start=7)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through the dataframe and add markers to the cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        icon=folium.Icon(color=row["marker_color"]),  # Use 'color', which is compatible with folium
        tooltip=f"Site: {row['Launch Site']}<br>Class: {row['class']}"
    ).add_to(marker_cluster)

# Display the map
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

## Distance of Launch Site CCAFS SLC-40 FROM THE COAST


In [30]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)


from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Approximate radius of Earth in kilometers
    R = 6373.0

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Differences in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

# Launch site coordinates
lat1, lon1 = 28.563197, -80.576820

# Coastline coordinates
lat2, lon2 = 28.56354, -80.56797

# Calculate the distance
distance_km = calculate_distance(lat1, lon1, lat2, lon2)
distance_km


0.865414876503625

In [31]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Initialize a folium map centered at the launch site
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a marker for the launch site
folium.Marker(
    location=[lat1, lon1],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(site_map)

# Add a marker for the coastline point, displaying the distance
folium.Marker(
    location=[lat2, lon2],
    popup=f"Coastline Point<br>Distance: {distance_km:.2f} km",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(site_map)

# Add a straight line (polyline) from the launch site to the coastline
folium.PolyLine(
    locations=[[lat1, lon1], [lat2, lon2]],
    color="red",
    weight= 2.0,
    opacity=1
).add_to(site_map)

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

# Add a marker with the manually set distance at the midpoint of the line
midpoint_lat = (lat1 + lat2) / 2
midpoint_lon = (lon1 + lon2) / 2

folium.Marker(
    location=[midpoint_lat, midpoint_lon],
    popup="Approx. Distance: 0.9 km",  # Manually set the displayed distance
    icon=folium.DivIcon(html=f"""<div style="font-size: 12px; color: blue;">0.9 km</div>""")
).add_to(site_map)

# Display the updated map with the midpoint marker
site_map




*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [33]:

# Initialize the map centered at the launch site
launch_site_coords = [28.562302, -80.577356]
site_map = folium.Map(location=launch_site_coords, zoom_start=12)


def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance
    
# Define the function to add a marker and a polyline
def add_marker_and_line(map_object, lat, lon, label, color, distance=None):
    # Add a marker for the point of interest
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=color),
        tooltip=f"{label}: {distance:.2f} km"
    ).add_to(map_object)
    
    # Add a polyline from the point of interest to the launch site
    folium.PolyLine(
        locations=[launch_site_coords, [lat, lon]],
        color=color,
        weight=2,
        tooltip=f"Distance: {distance:.2f} km"
    ).add_to(map_object)

# Calculate distances to the points of interest
city_coords = [28.07808, -80.6045]
railway_coords = [28.42936, -80.7653]
highway_coords = [28.45888, -80.54821]

city_distance = geodesic(launch_site_coords, city_coords).kilometers
railway_distance = geodesic(launch_site_coords, railway_coords).kilometers
highway_distance = geodesic(launch_site_coords, highway_coords).kilometers

# Add markers and lines for each point of interest
add_marker_and_line(site_map, city_coords[0], city_coords[1], "Closest City", "purple", city_distance)
add_marker_and_line(site_map, railway_coords[0], railway_coords[1], "Closest Railway", "orange", railway_distance)
add_marker_and_line(site_map, highway_coords[0], highway_coords[1], "Closest Highway", "red", highway_distance)

# Add a marker for the launch site itself
folium.Marker(
    location=launch_site_coords,
    icon=folium.Icon(color="blue", icon="rocket"),
    tooltip="Launch Site"
).add_to(site_map)

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.d_to(site_map)ap
